**Name:** YANG Yongze

**EID:** 58162280

**Kaggle Team Name:** YANG Yongze

# CS5489 - Assignment 2 - Urban Sound Tagging
Due date: see Assignment 2 on Canvas

# FINAL SOLUTION

In [2]:
%matplotlib inline
import matplotlib_inline   # setup output image format
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
random.seed(100)
import csv
from scipy import io
import pickle
from IPython.display import Audio, display

In [3]:
def showAudio(info):
    myfile = 'musicmp3/' + info['fname'] + '.mp3'
    if os.path.exists(myfile):
        display(Audio(myfile))
    else:
        print("*** mp3 file " + myfile + " could not be found ***")

def load_pickle(fname):
    f = open(fname, 'rb')
    out = pickle.load(f)
    f.close()
    return out

In [5]:
train_tags  = load_pickle('sounddata/train_tags.pickle3')
train_info  = load_pickle('sounddata/train_info.pickle3')
test_info  = load_pickle('sounddata/test_info.pickle3')

In [6]:
alltagnames, alltagnames_counts = unique(concatenate(train_tags), return_counts=True)
for a,b in zip(alltagnames, alltagnames_counts):
    print("{}: {}".format(a, b))

# remove some tags
blacklist = ['2-3_hoe-ram_presence', '2-4_pile-driver_presence', '5-2_car-alarm_presence', '5-X_other-unknown-alert-signal_presence', '6-3_ice-cream-truck_presence', '7-3_large-crowd_presence', '7-X_other-unknown-human-voice_presence']
tagnames = [t for t in alltagnames if t not in blacklist]
print(tagnames)

1-1_small-sounding-engine_presence: 193
1-2_medium-sounding-engine_presence: 616
1-3_large-sounding-engine_presence: 775
1-X_engine-of-uncertain-size_presence: 307
1_engine_presence: 1492
2-1_rock-drill_presence: 264
2-2_jackhammer_presence: 179
2-3_hoe-ram_presence: 79
2-4_pile-driver_presence: 55
2-X_other-unknown-impact-machinery_presence: 255
2_machinery-impact_presence: 650
3-1_non-machinery-impact_presence: 349
3_non-machinery-impact_presence: 349
4-1_chainsaw_presence: 101
4-2_small-medium-rotating-saw_presence: 100
4-3_large-rotating-saw_presence: 96
4-X_other-unknown-powered-saw_presence: 79
4_powered-saw_presence: 267
5-1_car-horn_presence: 286
5-2_car-alarm_presence: 28
5-3_siren_presence: 174
5-4_reverse-beeper_presence: 175
5-X_other-unknown-alert-signal_presence: 70
5_alert-signal_presence: 635
6-1_stationary-music_presence: 84
6-2_mobile-music_presence: 26
6-3_ice-cream-truck_presence: 6
6-X_music-from-uncertain-source_presence: 73
6_music_presence: 150
7-1_person-or-sma

In [7]:
# convert list of tags into binary class labels
def tags2class(tags, tagnames):
    b = zeros(shape=(len(tags), len(tagnames)))
    for i,t in enumerate(tags):
        for j,n in enumerate(tagnames):
            if n in t:
                b[i,j] = 1
    return b

In [8]:
# train_classes[i,j] = absence/presence of the j-th tag in the i-th sound
train_classes = tags2class(train_tags, tagnames)

In [9]:
import csv

def write_csv_kaggle_tags(fname, tagnames, Yscores):
    # header
    tmp = [['Id']]
    for t in tagnames:
        tmp[0].append(t)    
    
    # add ID numbers for each Y, and usage if necessary
    for i in range(len(Yscores)):
        tmp2 = [(i+1)]
        for t in range(len(tagnames)):
            tmp2.append(Yscores[i,t])
        
        tmp.append(tmp2)
        
    # write CSV file
    f = open(fname, 'w')
    writer = csv.writer(f)
    writer.writerows(tmp)
    f.close()

## YOUR CODE and DOCUMENTATION HERE

My method requires a python package named `pyAudioProcessing`, using the following cell to install.

In [ ]:
!pip install pyAudioProcessing

Now import this package:

In [ ]:
from pyAudioProcessing.extract_features import get_features

Using this package, we can process the raw wav data and get the feature representation of these wav files.

In [ ]:
train_concat = []
test_concat = []

for info in train_info:
    file_path = os.path.join("soundwav",info['fname']+'.wav')
    feature = get_features(file=file_path,feature_names=["gfcc","spectral","chroma"])
    feature_values = array(feature['audio'][file_path]['features'])
    train_concat.append(feature_values)

for info in test_info:
    file_path = os.path.join("soundwav",info['fname']+'.wav')
    feature = get_features(file=file_path,feature_names=["gfcc","spectral","chroma"])
    feature_values = array(feature['audio'][file_path]['features'])
    test_concat.append(feature_values)

train_concat = array(train_concat)
test_concat = array(test_concat)

print(train_concat.shape)
print(test_concat.shape)

with open('train_concat.pickle', 'wb') as f:
    pickle.dump(train_concat, f)

with open('test_concat.pickle', 'wb') as f:
    pickle.dump(test_concat, f)

It is OK if the package is not installed. In this case please load my pickle file.

In [10]:
train_concat = load_pickle('train_concat.pickle')
test_concat = load_pickle('test_concat.pickle')

Now use Grid Search to train an MLP classifier for each tag, and make the prediction.

In [11]:
def mlp_predict(train_feature, test_feature, csv_path='predict_mlp.csv'):
    parameter_space = {
        'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,), (100, 10), (50, 50)],
        'alpha': [0.0001, 0.05],
        'learning_rate': ['constant', 'invscaling','adaptive'],
    }
    
    tagmodels = {}
    for i,t in enumerate(tagnames):
        print('training {} - {}'.format(i, t))
        myY = myY = train_classes[:,i].ravel()
        mlp = model_selection.GridSearchCV(neural_network.MLPClassifier(), parameter_space, cv=5, n_jobs=-1, verbose=False)
        mlp.fit(train_feature, myY)
        tagmodels[t] = mlp

    test_predscore = zeros(shape=(test_feature.shape[0], len(tagnames)))

    for i,t in enumerate(tagnames):
        print('predicting {} - {}'.format(i, t))
        tmp = tagmodels[t].predict_proba(test_feature)
        test_predscore[:,i] = tmp[:, 1]

    write_csv_kaggle_tags(csv_path, tagnames, test_predscore)

In [12]:
mlp_predict(train_concat, test_concat, csv_path='final_submission.csv')

training 0 - 1-1_small-sounding-engine_presence


c:\Users\yyz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


training 1 - 1-2_medium-sounding-engine_presence


c:\Users\yyz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


training 2 - 1-3_large-sounding-engine_presence
training 3 - 1-X_engine-of-uncertain-size_presence


...

c:\Users\yyz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


training 26 - 7-4_amplified-speech_presence
training 27 - 7_human-voice_presence
training 28 - 8-1_dog-barking-whining_presence


c:\Users\yyz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


training 29 - 8_dog_presence
predicting 0 - 1-1_small-sounding-engine_presence
predicting 1 - 1-2_medium-sounding-engine_presence
predicting 2 - 1-3_large-sounding-engine_presence
predicting 3 - 1-X_engine-of-uncertain-size_presence
predicting 4 - 1_engine_presence
predicting 5 - 2-1_rock-drill_presence
predicting 6 - 2-2_jackhammer_presence
predicting 7 - 2-X_other-unknown-impact-machinery_presence
predicting 8 - 2_machinery-impact_presence
predicting 9 - 3-1_non-machinery-impact_presence
predicting 10 - 3_non-machinery-impact_presence
predicting 11 - 4-1_chainsaw_presence
predicting 12 - 4-2_small-medium-rotating-saw_presence
predicting 13 - 4-3_large-rotating-saw_presence
predicting 14 - 4-X_other-unknown-powered-saw_presence
predicting 15 - 4_powered-saw_presence
predicting 16 - 5-1_car-horn_presence
predicting 17 - 5-3_siren_presence
predicting 18 - 5-4_reverse-beeper_presence
predicting 19 - 5_alert-signal_presence
predicting 20 - 6-1_stationary-music_presence
predicting 21 - 6-2

c:\Users\yyz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Now we have the final submission csv file `final_submission.csv` in the current path.

This subnmission has a public score of 0.82379 in the Kaggle Competition and ranks 19/137 for now (21:13, 6/11/2023).